In [3]:
from jinja2 import Template

from copy import deepcopy

oil_template = Template("""
OIL_VERSION = "2.5" : "test" ;

CPU test {
  OS config {
    STATUS = STANDARD;
    BUILD = TRUE {
      TRAMPOLINE_BASE_PATH = "../trampoline";
      APP_NAME = "avr-AUTOSAR-trampoline-Blink.elf";
      APP_SRC = "avr-AUTOSAR-trampoline-Blink.cpp";
      CPPCOMPILER = "avr-g++";
      COMPILER = "avr-gcc";
      LINKER = "avr-gcc";
      ASSEMBLER = "avr-gcc";
      COPIER = "avr-objcopy";
      SYSTEM = PYTHON;
      LIBRARY = serial;
    };
    SYSTEM_CALL = TRUE;
  };

  APPMODE stdAppmode {};

{% for task in tasks %}
  ALARM {{ task.name }}Alarm {
    COUNTER= SystemCounter;
    ACTION = ACTIVATETASK {
      TASK = {{ task.name }};
    };
    AUTOSTART = TRUE {
      ALARMTIME = {{ task.alarmtime }};
      CYCLETIME = {{ task.cycletime }};
      APPMODE = stdAppmode;
    };
  };
  TASK {{ task.name }} {
    PRIORITY = {{ task.priority }};
    AUTOSTART = FALSE;
    ACTIVATION = 1;
    SCHEDULE = FULL;
    STACKSIZE = {{ task.stacksize }};
  };
{% endfor %}

};
""")

In [13]:
tasks = list()
task_default = {
    "alarmtime": 5000,
    "cycletime": 10,
    "priority": 20,
    "stacksize": 256,
    "c_type": "unsigned char"
}

cycletime=10
for pin in range(8, 14):
    task = deepcopy(task_default)
    task["name"]=f"blinkTaskPin{pin}"
    task["pin"]=pin
    task["cycletime"]=cycletime
    tasks.append(task)
    cycletime=cycletime*2
with open("avr-AUTOSAR-trampoline-Blink.oil", "w") as fid:
    print(oil_template.render(tasks=tasks), file=fid)

In [46]:
cpp_template = Template("""
#include "Arduino.h"
#undef ISR // remove the avr-gcc ISR version
extern "C" {
#include "tpl_app_config.h"
#include "tpl_os.h" //set the OSEK ISR version.
}

void setup() {
{% for task in tasks %}
  pinMode({{ task.pin }}, OUTPUT);
{% endfor %}

  // "Ground" for Interrupts.
  pinMode(4, OUTPUT);
  digitalWrite(4, LOW);

  // Serial.
  Serial.begin(115200);
}

{% for task in tasks %}
TASK({{ task.name }}) {
  static {{ task.c_type }} nb = 0;
  digitalWrite({{ task.pin }}, nb++ & 0x01);
}
{% endfor %}
""")
with open("avr-AUTOSAR-trampoline-Blink.cpp", "w") as fid:
    print(cpp_template.render(tasks=tasks), file=fid)
!make

python3 make.py all
Nothing to make.
[ 50%] Compiling avr-AUTOSAR-trampoline-Blink.cpp
[100%] Linking avr-AUTOSAR-trampoline-Blink.elf
       Generating binary avr-AUTOSAR-trampoline-Blink.elf.hex from avr-AUTOSAR-trampoline-Blink.elf
avr-size avr-AUTOSAR-trampoline-Blink.elf.hex > avr-AUTOSAR-trampoline-Blink.elf.hex.size
avr-size avr-AUTOSAR-trampoline-Blink.elf > avr-AUTOSAR-trampoline-Blink.elf.size


In [47]:
with open("avr-AUTOSAR-trampoline-Blink.elf.hex.size", "r") as fid:
    data = fid.read()

In [48]:
import subprocess

def make(*args):
    return subprocess.check_output(["make", *args]).decode("UTF-8")

make()

In [51]:
!cat avr-AUTOSAR-trampoline-Blink.elf.hex.size

   text	   data	    bss	    dec	    hex	filename
      0	   8024	      0	   8024	   1f58	avr-AUTOSAR-trampoline-Blink.elf.hex


In [52]:
keys, values, _= data.split("\n")

In [53]:
hex_size = dict()
for key, value in zip(keys.split("\t"), values.split("\t")):
    key = key.strip()
    value = value.strip()
    hex_size[key]=value

In [54]:
hex_size

{'text': '0',
 'data': '8024',
 'bss': '0',
 'dec': '8024',
 'hex': '1f58',
 'filename': 'avr-AUTOSAR-trampoline-Blink.elf.hex'}